# 04. Data Processing

### Importando as bibliotecas

In [7]:
import sys
import warnings
sys.path.append('..')
import pandas as pd
import functions.fn_stats as fn_stats
import functions.fn_charts as fn_charts
import params.consts as consts

### Ignorando os warnings irrelevantes

Ignorando os warnings indevidos que apareceram devido a cópia de dataframes para fazer ajustes nos dados. 

Obs: na primeira vez que fiz o projeto não houve esses avisos não, mas agora melhorando alguns pontos apareceu do nada. Tentei retirar mas continua dando esses avisos.

In [8]:
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning) # Ignorando os warnings do tipo SettingWithCopyWarning

### Armazenando os datasets em uma variável

In [9]:
df_movies = pd.read_csv(consts.DATASET_MOVIES, low_memory=False) # Armazenando o dataset em uma variável

df_ratings = pd.read_csv(consts.DATASET_RATINGS, low_memory=False) # Armazenando o dataset em uma variável

### Excluindo as colunas que contém valores únicos
- N/A

### Excluindo as colunas que contém valores constantes
- N/A

### Excluindo as colunas que contém valores vazios
- N/A

### Excluindo as colunas que contém muitos valores vazios
- N/A

### Excluindo as colunas que contém informações redundantes
- N/A

### Excluindo as colunas que contém informações irrelevantes
- N/A

In [10]:
df_movies = df_movies.drop([
    'adult',
    'belongs_to_collection',
    'budget',
    'genres',
    'homepage',
    'imdb_id',
    'overview',
    'popularity',
    'poster_path',
    'production_companies',
    'production_countries',
    'release_date',
    'revenue',
    'runtime',
    'spoken_languages',
    'status',
    'tagline',
    'title',
    'video',
    'vote_average',
    ], 
    axis=1 # Definindo que as colunas serão excluídas
) # Excluindo as colunas

In [11]:
df_ratings = df_ratings.drop([
    'timestamp',
    ], 
    axis=1 # Definindo que as colunas serão excluídas
) # Excluindo as colunas

### Excluindo as colunas que contém textos livres
- N/A

### Lendo os datasets tratados e visualizando o overview atual

In [12]:
pd.set_option('display.max_columns', None) # Alterando as configurações do Pandas para exibir mais de 20 colunas

display(df_movies.head(3)) # Exibindo as 3 primeiras linhas do dataset

display(df_ratings.head(3)) # Exibindo as 3 primeiras linhas do dataset

,id,original_language,original_title,vote_count
0,862,en,Toy Story,5415.0
1,8844,en,Jumanji,2413.0
2,15602,en,Grumpier Old Men,92.0


,userId,movieId,rating
0,1,110,1.0
1,1,147,4.5
2,1,858,5.0


In [13]:
display(df_movies.shape) # Visualizando a quantidade de linhas e colunas do dataset

display(df_ratings.shape) # Visualizando a quantidade de linhas e colunas do dataset

(45466, 4)

(26024289, 3)

In [14]:
display(df_movies.info()) # Exibindo as informações das variáveis do dataset

display(df_ratings.info()) # Exibindo as informações das variáveis do dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 45466 non-null  object 
 1   original_language  45455 non-null  object 
 2   original_title     45466 non-null  object 
 3   vote_count         45460 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.4+ MB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26024289 entries, 0 to 26024288
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 595.6 MB


None

In [15]:
display(fn_stats.describe(df_movies)) # Usando a função que exibe as estatísticas das colunas numéricas

display(fn_stats.describe(df_ratings)) # Usando a função que exibe as estatísticas das colunas numéricas

,vote_count
count,45460.00
mean,109.90
std,491.31
min,0.00
25%,3.00
50%,10.00
75%,34.00
max,14075.00


None

,userId,movieId,rating
count,26024289.00,26024289.00,26024289.00
mean,135037.09,15849.11,3.53
std,78176.20,31085.26,1.07
min,1.00,1.00,0.50
25%,67164.00,1073.00,3.00
50%,135163.00,2583.00,3.50
75%,202693.00,6503.00,4.00
max,270896.00,176275.00,5.00


None

In [16]:
display(df_movies.describe(exclude='number')) # Exibindo as estatísticas das colunas categóricas

,id,original_language,original_title
count,45466,45455,45466
unique,45436,92,43373
top,141971,en,Hamlet
freq,3,32269,8


In [17]:
display(df_movies.isnull().sum()) # Somando todos os valores nulos de cada coluna

display(df_ratings.isnull().sum()) # Somando todos os valores nulos de cada coluna

id                    0
original_language    11
original_title        0
vote_count            6
dtype: int64

userId     0
movieId    0
rating     0
dtype: int64

### Tratando as colunas com valores nulos/vazios

In [25]:
df_movies = df_movies.dropna(subset=[
    'original_language',
    'vote_count',
]) # Removendo as linhas com valores nulos da coluna pois são poucos registros em relação à todo o dataset

### Corrigindo os tipos de dados das colunas

In [ ]:
df['Dt_Customer'] = pd.to_datetime(df['Dt_Customer'], format='%Y-%m-%d') # Convertendo a coluna para o formato de data

### Criando colunas de tempo/idade

In [ ]:
df['Days_Since_Enrolled'] = df['Dt_Customer'].max() - df['Dt_Customer'] # Criando a coluna com a diferença do valor máximo das datas e a data de cadastro do cliente
df['Days_Since_Enrolled'] = df['Days_Since_Enrolled'].dt.days # Convertendo o tempo para dias
df['Years_Since_Enrolled'] = df['Days_Since_Enrolled'] // 365 # Criando a coluna apenas com anos inteiros (dividindo com 2 barras)
df['Age'] = df['Dt_Customer'].max().year - df['Year_Birth'] # Criando a coluna com a idade dos clientes, fixando o valor máximmo das datas da campanha para que as idades não fiquem variando de acordo com o tempo que o código é rodado

### Mesclando colunas com muitas categorias

In [ ]:
df['AgeGroup'] = pd.cut( # Criando essa coluna para agrupar as idades em intervalos padrões de cerca de 15 anos
    df['Age'], # Definindo a coluna que será usada para criar os intervalos
    bins=[18, 30, 45, 60, df['Age'].max()], # Definindo os intervalos
    labels=['18-30', '31-45', '46-60', '61+'], # Definindo os rótulos de como os dados vão aparecer no dataset
    include_lowest=True # Incluindo o valor inicial no intervalo
)
df['Marital_Status'] = df['Marital_Status'].replace( # Agrupando melhor os dados dessa coluna
    {
        'Alone': 'Single',
        'Absurd': 'Single',
        'YOLO': 'Single',
        'Together': 'Partner',
        'Married': 'Partner',
        'Widow': 'Single',
        'Single': 'Single',
        'Divorced': 'Single',
    }
)

In [ ]:
display(df['AgeGroup'].value_counts(normalize=True).map('{:.1%}'.format).sort_index()) # Verificando o resultado da transformação na coluna em %
display(df['Marital_Status'].value_counts(normalize=True).map('{:.1%}'.format).sort_index()) # Verificando o resultado da transformação na coluna em %

### Unificando colunas semelhantes

In [ ]:
df['Children'] = df['Kidhome'] + df['Teenhome'] # Criando a coluna com o total de dependentes do cliente
df['MntTotal'] = ( # Criando a coluna com o total gasto em todas as categorias de produtos
    df['MntWines'] + 
    df['MntFruits'] + 
    df['MntMeatProducts'] + 
    df['MntFishProducts'] + 
    df['MntSweetProducts'] + 
    df['MntGoldProds']
)
df['AcceptedCmpTotal'] = ( # Criando a coluna com o total de campanhas aceitas pelo cliente
    df['AcceptedCmp1'] + 
    df['AcceptedCmp2'] + 
    df['AcceptedCmp3'] + 
    df['AcceptedCmp4'] + 
    df['AcceptedCmp5'] 
)
df['NumTotalPurchases'] = ( # Criando a coluna com o total de compras feitas pelo cliente
    df['NumWebPurchases'] + 
    df['NumCatalogPurchases'] + 
    df['NumStorePurchases'] 
)

In [ ]:
display(df['Children'].value_counts(normalize=True).map('{:.1%}'.format).sort_index()) # Verificando o resultado da transformação na coluna em %
display(df['AcceptedCmpTotal'].value_counts(normalize=True).map('{:.1%}'.format).sort_index()) # Verificando o resultado da transformação na coluna em %

### Criando colunas derivadas de outras

In [ ]:
df['HasChildren'] = df['Children'].apply(lambda x: 1 if x > 0 else 0) # Criando a coluna que verifica se o cliente tem dependentes através de uma função anônima (na própria linha de código)
df['MntRegularProds'] = df['MntTotal'] - df['MntGoldProds'] # Criando a coluna com o valor gasto somente em produtos regulares
df['HasAcceptedCmp'] = df['AcceptedCmpTotal'].apply(lambda x: 1 if x > 0 else 0) # Criando a coluna que verifica se o cliente já aceitou uma campanha através de uma função anônima (na própria linha de código)

### Visualizando o dataset antes de tratar outliers e excluir colunas

In [ ]:
df.head(3) # Exibindo as 3 primeiras linhas do dataset

### Tratando as colunas com valores outliers

In [ ]:
fn_charts.boxplots(df,  ['Age', 'Income', 'MntTotal']) # Exibindo os gráficos de boxplots antes do tratamento

In [ ]:
fn_stats.inspect_outliers(df, 'Age') # Usando a função para listar os outliers

In [ ]:
fn_stats.inspect_outliers(df, 'Income') # Usando a função para listar os outliers

In [ ]:
df = df.drop(fn_stats.inspect_outliers(df,'Age').index) # Removendo os outliers 
df = df.drop(fn_stats.inspect_outliers(df,'Income').index) # Removendo os outliers 

In [ ]:
fn_charts.boxplots(df,  ['Age', 'Income', 'MntTotal']) # Exibindo os gráficos de boxplots antes do tratamento

### Salvando o dataset processado

In [ ]:
df.to_csv(consts.DATASET_PROCESSED, index=False) # Salvando o dataset processado

### Excluindo as colunas auxiliares

In [ ]:
df = df.drop( # Excluindo as colunas auxiliares
    [ 
        'Dt_Customer', 
        'Year_Birth', 
        'Kidhome',
        'Teenhome',
        'MntWines', 
        'MntFruits', 
        'MntMeatProducts', 
        'MntFishProducts', 
        'MntSweetProducts',
        'AcceptedCmp1',
        'AcceptedCmp2',
        'AcceptedCmp3',
        'AcceptedCmp4',
        'AcceptedCmp5', 
        'NumWebPurchases', 
        'NumCatalogPurchases', 
        'NumStorePurchases', 
    ], 
    axis=1 # Definindo que colunas serão excluídas
) 

### Ordenando as colunas relacionadas

In [ ]:
df = df.reindex(columns=[ # Ordenando as colunas em uma ordem que faz mais sentido para análises
        'Education', 
        'Marital_Status', 
        'Children',
        'HasChildren',
        'Age',
        'AgeGroup',
        'Income',
        'Recency',
        'Complain',
        'Days_Since_Enrolled',
        'Years_Since_Enrolled',
        'NumDealsPurchases',
        'NumWebVisitsMonth',
        'NumTotalPurchases',
        'MntRegularProds',
        'MntGoldProds',
        'MntTotal',
        'AcceptedCmpTotal',
        'HasAcceptedCmp',
        'Response',
    ]
)

### Salvando o dataset lean

In [ ]:
df.to_csv(consts.DATASET_LEAN, index=False) # Salvando o dataset lean